In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW tv_orders_exploded as
SELECT 
  json_value.order_id AS order_id, 
  json_value.customer_id AS customer_id, 
  json_value.order_status, 
  json_value.payment_method, 
  json_value.total_amount, 
  cast(json_value.transaction_timestamp as date) AS transaction_date,
  date_format(json_value.transaction_timestamp, 'HH:mm:dd') AS transaction_time,
  explode(array_distinct(json_value.items)) AS item
FROM 
  gizmo_box.silver.orders_json;

In [0]:
%sql
SELECT 
  order_id, 
  customer_id, 
  order_status, 
  payment_method, 
  total_amount, 
  transaction_date, 
  transaction_time, 
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
FROM 
  tv_orders_exploded;

In [0]:
%sql

-- write to silver

CREATE TABLE IF NOT EXISTS gizmo_box.silver.orders as 
SELECT 
  order_id, 
  customer_id, 
  order_status, 
  payment_method, 
  total_amount, 
  transaction_date, 
  transaction_time, 
  item.item_id,
  item.name,
  item.price,
  item.quantity,
  item.category,
  item.details.brand,
  item.details.color
FROM 
  tv_orders_exploded;